In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import platform

In [ ]:
air_I = pd.read_csv('air.csv',encoding='euc_kr',header=0)
air_I = pd.DataFrame(air_I)
air_I = air_I[air_I['결항여부'] != 1]  # 결항 데이터들 삭제 .. 결항 데이터는 지연에 포함시키지 않음. 
air_I = air_I[air_I['부정기편'] != 1]  # 부정기편 데이터들 삭제 .. 부정기편은 정규 편성된 항공편에서 예외로 생겨났던 것들이기 때문에
# 예측 변수로 사용할 수 없기 때문에 삭제하였다.

air_I.head()

In [ ]:
air_I.shape   ## 93558개의 데이터 , 21개의 변수

In [ ]:
air_I.isnull().values.any()  # 결측치 존재 여부

In [ ]:
air_I.isnull().sum()  # 결측치 갯수  

In [ ]:
# 변수 선택하기

air_I = air_I[["공항","항공사","편명","출도착","시간대","day_airline","VIS","평균풍속(kts)","지연여부"]]


In [ ]:
# 더미 변수
air_I = pd.get_dummies(air_I, columns=['공항',"항공사","편명"])
air_I.head()

In [ ]:
# 훈련집합, 평가집합 나누기
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(air_I.drop('지연여부', axis=1), air_I['지연여부'], test_size=0.2, random_state=42)

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
''
# 데이터 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit_transform(train_x)
train_x = scaler.fit_transform(train_x)


In [ ]:
train_x

In [ ]:

# 데이터 복제
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# 모델설정  SMOTE 기법을 사용한 Over Sampling
sm = SMOTE()
#sm = SMOTE(ratio='auto', kind='regular')
# train데이터를 넣어 복제함
X_resampled, y_resampled = sm.fit_sample(train_x,list(train_y))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))




In [ ]:
X_resampled

In [ ]:
y_resampled

In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=13)
model.fit(train_x, train_y)


In [ ]:
# predict 메서드를 호출하여 test_x의 값으로 모델을 테스트한 다음, 
# score 메서드를 사용하여 모델의 평균 정확도를 확인합니다.
predicted = model.predict(test_x)
model.score(test_x, test_y)

In [ ]:
# 평가집합에서 예측 확률 세트를 생성
from sklearn.metrics import roc_auc_score
probabilities = model.predict_proba(test_x)

In [ ]:
# roc auc 점수 생성 
# 데이터의 불균형으로 인해 항공편의 지연 도착 예측보다 항공편의 정시 도착 예측이 정확할 가능성이 더 높습니다.
# ROC AUC는 이를 고려하며, 정시 또는 지연 도착에 대한 예측이 정확할 가능성을 더 정확하게 나타냅니다.
roc_auc_score(test_y, probabilities[:, 1])

In [ ]:
# 혼동 행렬 생성 - 혼동 행렬은 각 답변이 정확하거나 잘못 분류된 횟수를 정량화합니다. 
# 특히 거짓 긍정, 거짓 부정, 참 긍정 및 참 부정의 수를 정량화합니다. 
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predicted)

In [ ]:
# 정밀도 계산 - precision score
from sklearn.metrics import precision_score

train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions)

In [ ]:
# 회수 계산용 -recall score
from sklearn.metrics import recall_score

recall_score(train_y, train_predictions)

In [ ]:
### 모델 출력 시각화
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
# ROC 곡선
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probabilities[:, 1])
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('거짓 긍정 비율')
plt.ylabel('참 긍정 비율')